In [23]:
import sys
import yaml

BASE_CONFIG_FILE = "base_dispatcher_config.yaml"
CONTRACTS_DEPLOYMENT_INFO_FILE = "deployed_dispute_contracts.yaml"
OUTPUT_CONFIG_FILE = "dispatcher_config.yaml"
BASE_ABI_PATH = "/opt/cartesi/blockchain/abis/"

def replace_deployed_info_in_base_config(conf_yaml, dep_info_yaml):
    conf_yaml['concerns']=[]
    
    for concern in dep_info_yaml['concerns']:
        replaced_concern = {}
        
        replaced_concern['user_address'] = concern['user_address']
        replaced_concern['contract_address'] = concern['contract_address']
        
        abi_name = concern['abi'].split('/')[-1]
        replaced_concern['abi'] = BASE_ABI_PATH + abi_name
        
        #Checking if it is the main concern or not
        if abi_name == "ComputeInstantiator.json":
            conf_yaml['main_concern'] = replaced_concern
        else:
            conf_yaml['concerns'].append(replaced_concern)
    return conf_yaml

def load_contracts_deployment_info():
    with open(CONTRACTS_DEPLOYMENT_INFO_FILE, 'r') as contracts_deployment_info_file:
        try:
            return yaml.safe_load(contracts_deployment_info_file)
        except yaml.YAMLError as exc:
            print("Error trying to load contracts deployment info yaml:")
            print(exc)
            sys.exit(1)        

def write_dynamic_confs(conf_yaml):
    dep_info_yaml = load_contracts_deployment_info()
    return replace_deployed_info_in_base_config(conf_yaml, dep_info_yaml)

loaded_yaml = None
custom_yaml = None

with open(BASE_CONFIG_FILE, 'r') as base_yaml_file:
    try:
        loaded_yaml = yaml.safe_load(base_yaml_file)
    except yaml.YAMLError as exc:
        print("Error trying to load yaml base configuration:")
        print(exc)
        sys.exit(1)
        
    custom_yaml = write_dynamic_confs(loaded_yaml)
    
    print("Dumping final configuration to file:\n{}".format(yaml.dump(custom_yaml, default_flow_style=False)))
    
    with open(OUTPUT_CONFIG_FILE, 'w') as custom_yaml_file:
        yaml.dump(custom_yaml, custom_yaml_file, default_flow_style=False)



Dumping final configuration to file:
concerns:
- abi: /opt/cartesi/blockchain/abis/PartitionInstantiator.json
  contract_address: '0x5831eD055B436464ce915c06E2146d7f0bc055DA'
  user_address: '0x2AD38F50f38Abc5CbcF175e1962293EEcc7936De'
- abi: /opt/cartesi/blockchain/abis/MMInstantiator.json
  contract_address: '0x9758599Dd9E507322F55784811A6A72bDd2443d9'
  user_address: '0x2AD38F50f38Abc5CbcF175e1962293EEcc7936De'
- abi: /opt/cartesi/blockchain/abis/VGInstantiator.json
  contract_address: '0x63664106a027195f368B8ac0e20C460f95e570C4'
  user_address: '0x2AD38F50f38Abc5CbcF175e1962293EEcc7936De'
confirmations: 0
emulator_port: 50052
main_concern:
  abi: /opt/cartesi/blockchain/abis/ComputeInstantiator.json
  contract_address: '0xbE35F0971d377C0CC290288539434E6D4C13cc7d'
  user_address: '0x2AD38F50f38Abc5CbcF175e1962293EEcc7936De'
max_delay: 500
query_port: 3002
testing: true
url: http://127.0.0.1:8545
warn_delay: 30

